# Sissejuhatus tõenäosusteooriasse ja statistikasse
Selles märkmikus mängime mõningate varasemalt käsitletud mõistetega. Paljud tõenäosuse ja statistika kontseptsioonid on hästi esindatud suurtes Python'i andmetöötlusteekides, nagu `numpy` ja `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Juhuslikud muutujad ja jaotused
Alustame 30 väärtuse proovivõtmisega ühtlasest jaotusest 0 kuni 9. Samuti arvutame keskmise ja dispersiooni.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Selleks, et visuaalselt hinnata, kui palju erinevaid väärtusi proovis on, saame joonistada **histogrammi**:


In [ ]:
plt.hist(sample)
plt.show()

## Tõelise andmete analüüs

Keskmine ja dispersioon on tõeliste andmete analüüsimisel väga olulised. Laadime andmed pesapallurite kohta saidilt [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Me kasutame siin andmete analüüsimiseks paketti [**Pandas**](https://pandas.pydata.org/). Me räägime pandast ja andmetega töötamisest Pythonis selle kursuse edasistes osades rohkem.

Arvutame keskmised väärtused vanuse, pikkuse ja kaalu jaoks:


In [ ]:
df[['Age','Height','Weight']].mean()

Nüüd keskendume pikkusele ja arvutame standardhälbe ning dispersiooni:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Keskmise kõrval on mõistlik vaadata ka mediaani väärtust ja kvartiile. Neid saab visualiseerida kasutades **kastdiagrammi**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Me saame teha ka kastdiagramme meie andmekogu alamhulkadest, näiteks mängijate rolli järgi rühmitatult.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Märkus**: See diagramm näitab, et keskmiselt on esimesed esimesed põhitõkkepedjad pikemad kui teised põhitõkkepedjad. Hiljem õpime, kuidas seda hüpoteesi formaalsemalt testida ja kuidas tõestada, et meie andmed on statistiliselt olulised selle näitamiseks.  

Vanus, pikkus ja kaal on kõik pidevad juhuslikud muutujad. Mida sa arvad, milline on nende jaotus? Hea viis selle väljaselgitamiseks on joonistada väärtuste histogramm: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normaaljaotus

Loon kunstliku masside valimi, mis järgib normaaljaotust sama keskmise ja dispersiooniga kui meie tegelikud andmed:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Kuna enamik elus olevatest väärtustest on normaalselt jaotunud, ei tohiks me kasutada ühtlast juhuslike arvude generaatorit proovandmete genereerimiseks. Siin on see, mis juhtub, kui proovime kaalusid genereerida ühtlase jaotusega (genereeritud `np.random.rand` abil):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Usaldusvahemikud

Arvutame nüüd pesapallimängijate kehakaalu ja pikkuse usaldusvahemikud. Kasutame koodi [selles stackoverflow arutelus](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Hüpoteesi testimine

Uurime erinevaid rolle meie pesapallimängijate andmestikus:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Testime hüpoteesi, et esimesed pesamehed on teistest pesameestest pikemad. Kõige lihtsam viis selleks on testida usaldusvahemikke:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Saame näha, et intervallid ei kattu.

Statistiliselt korreksem viis hüpoteesi tõestamiseks on kasutada **Studenti t-testi**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Funktsiooni `ttest_ind` tagastatud kaks väärtust on:
* p-väärtust võib pidada tõenäosuseks, et kaks jaotust omavad sama keskmist väärtust. Meie puhul on see väga madal, mis tähendab, et on tugevad tõendid selle kohta, et esimestel pesuritel on suurem pikkus.
* t-väärtus on normeeritud keskmise erinevuse vaheväratlus, mida kasutatakse t-testis, ja seda võrreldakse antud usaldusväärtuse künnisega.


## Normaalse jaotuse simuleerimine keskväärtusteooreemi abil

Juhuslike arvude generaator Pythonis on loodud tagama ühtlase jaotuse. Kui soovime luua normaaljaotusega generaatori, võime kasutada keskväärtusteooreemi. Normaaljaotusega väärtuse saamiseks arvutame lihtsalt ühtlaselt genereeritud valimi aritmeetilise keskmise.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korrelatsioon ja Evil Baseball Corp

Korrelatsioon võimaldab meil leida seoseid andmesarjade vahel. Meie mängulises näites kujutame ette, et on kuri pesapallifirma, kes maksab oma mängijatele vastavalt nende pikkusele – mida pikem on mängija, seda rohkem raha ta saab. Oletame, et algpalk on 1000 dollarit ning lisaks saab kõrgusest sõltuva boonuse 0 kuni 100 dollarit. Võtame MLB mängijad ja arvutame nende kujuteldavad palgad:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Lähme nüüd arvutama nende järjestuste kovariantsust ja korrelatsiooni. Funktsioon `np.cov` annab meile nn **kovariantsusmaatriksi**, mis on kovariantsuse laiendus mitmele muutujaile. Kovariantsusmaatriksi $M$ element $M_{ij}$ on sisendmuutujate $X_i$ ja $X_j$ omavaheline korrelatsioon ning diagonaalväärtused $M_{ii}$ on $X_i$ dispersioon. Samamoodi annab `np.corrcoef` meile **korrelatsioonimaatriksi**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Seos, mis on võrdne 1-ga, tähendab, et kahe muutuja vahel on tugev **lineaarne seos**. Lineaarse seose saab visuaalselt näha, kui ühe väärtuse teise vastu joonistada:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Vaatame, mis juhtub, kui seos ei ole lineaarne. Oletame, et meie ettevõte otsustas peita ilmset lineaarset sõltuvust pikkuste ja palkade vahel ning lisas valemisse mõningase mittelineaarsuse, näiteks `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Selles olukorras on korrelatsioon veidi väiksem, kuid siiski üsna kõrge. Nüüd, et seost veel vähem ilmseks teha, võime soovida palgale lisada natuke juhuslikkust, lisades mõne juhusliku muutuja. Vaatame, mis juhtub:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Kas suudad ära arvata, miks täpid reas ritta vertikaalseteks joonteks moodustuvad?

Oleme täheldanud korrelatsiooni kunstlikult loodud mõiste nagu palk ja täheldatud muutujate *pikkus* vahel. Vaatame ka, kas kaks täheldatud muutujat, nagu pikkus ja kaal, korreleeruvad:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Kahjuks me ei saanud ühtegi tulemust – ainult mõned kummalised `nan` väärtused. See on tingitud sellest, et mõned meie seeria väärtused on määramata, mida tähistatakse kui `nan`, mis põhjustab operatsiooni tulemuse samuti määramatuks olemise. Vaadates maatriksit näeme, et probleemne veerg on `Weight`, kuna kõrguse väärtuste omakorpus on arvutatud.

> See näide näitab **andmete ettevalmistamise** ja **puhastamise** tähtsust. Ilma korralike andmeteta ei saa me midagi arvutada.

Kasutame `fillna` meetodit, et täita puuduvad väärtused ja arvutame korrelatsiooni:


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Tõepoolest on olemas korrelatsioon, kuid mitte nii tugev kui meie kunstlikus näites. Kui vaadata ühte väärtust teise vastu hajuvusdiagrammil, oleks seos palju vähem ilmne:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Kokkuvõte

Selles märkmikus oleme õppinud, kuidas andmetel põhitoiminguid teha, et arvutada statistilisi funktsioone. Nüüd teame, kuidas kasutada matemaatika ja statistika põhjalikku aparatuuri, et mõningaid hüpoteese tõestada ning kuidas arvutada usaldusvahemikke suvalistele muutujatele, kui on olemas andmete valim.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vastutusest loobumine**:  
See dokument on tõlgitud tehisintellekti tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi me püüdleme täpsuse poole, palun arvestage, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Originaaldokument tema emakeeles peaks olema autoriteetne allikas. Tähtsate andmete puhul soovitatakse kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tekkida võivate arusaamatuste või valesti mõistmiste eest.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
